In [ ]:
import sys, platform
EXPECTED = ".venv"
assert EXPECTED in sys.executable, (
    f"  Kernel incorrecto ({sys.executable}).\n"
    "Activa el entorno y selecciona el kernel 'Python (NoCountry_74)'.\n\n"
    "Pasos rápidos:\n"
    "1️ source .venv/bin/activate\n"
    "2️ python -m ipykernel install --user --name nocountry74 --display-name 'Python (NoCountry_74)'\n"
    "3️ En el notebook: cambia el kernel a 'Python (NoCountry_74)'."
)
print(f" Kernel OK: {sys.executable}  |  Python {platform.python_version()}")


 Kernel OK: /workspaces/NoCountry_74/.venv/bin/python  |  Python 3.12.1


In [ ]:
# === Importaciones base para el EDA ===
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# === Cargar dataset ===
DATA_PATH = Path("../data/raw/abandono_bancario.csv")
assert DATA_PATH.exists(), f"No se encontró el archivo en {DATA_PATH.resolve()}"

df = pd.read_csv(DATA_PATH)

print("Shape (filas, columnas):", df.shape)
display(df.head(10))


Shape (filas, columnas): (10132, 21)


,Numero Cliente,Estado Abandono,Edad Cliente,Genero,Cant. Personas a Cargo,Nivel Educativo,Estado Civil,Rango Ingresos,Categoria Tarjeta,Antiguedad Cuenta,Total Productos Bancarios,Meses Inactivos Ultimo Año,Frecuencia de Contacto al Banco,Limite Credito,Saldo Pendiente Tarjeta,Promedio Credito Disponible,Cambio Monto Q4 Q1,Monto Total Transacciones,Numero Total Transacciones,Cambio Numero Transacciones Q4 Q1,Porcentaje Uso Credito
0,768805383,Cliente Existente,45.0,Masculino,3,Secundaria,Casado,$60K - $80K,Azul,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Cliente Existente,49.0,Femenino,5,Graduado,Soltero,Menos de $40K,Azul,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Cliente Existente,51.0,Masculino,3,Graduado,Casado,$80K - $120K,Azul,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Cliente Existente,40.0,Femenino,4,Secundaria,Desconocido,Menos de $40K,Azul,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Cliente Existente,NaN,Masculino,3,Sin Educación,Casado,$60K - $80K,Azul,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,713061558,Cliente Existente,44.0,Masculino,2,Graduado,Casado,$40K - $60K,Azul,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
6,810347208,Cliente Existente,51.0,Masculino,4,Desconocido,Casado,Mas de $120K,Oro,46,6,1,3,34516.0,2264,32252.0,1.975,1330,31,0.722,0.066
7,818906208,Cliente Existente,32.0,Masculino,0,Secundaria,Desconocido,$60K - $80K,Plata,27,2,2,2,29081.0,1396,27685.0,2.204,1538,36,0.714,0.048
8,710930508,Cliente Existente,NaN,Masculino,3,Sin Educación,Soltero,$60K - $80K,Azul,36,5,2,0,22352.0,2517,19835.0,3.355,1350,24,1.182,0.113
9,719661558,Cliente Existente,48.0,Masculino,2,Graduado,Soltero,$80K - $120K,Azul,36,6,3,3,11656.0,1677,9979.0,1.524,1441,32,0.882,0.144


In [ ]:
# === Perfil básico de columnas y tipos ===
print("=== df.info ===")
df.info(memory_usage='deep')

# Separar columnas por tipo
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(include=['object','category','bool']).columns.tolist()

print("\nNuméricas:", len(num_cols), num_cols)
print("\nCategóricas:", len(cat_cols), cat_cols)

# Cardinalidad de categóricas (ayuda a detectar IDs que no conviene one-hot)
print("\nCardinalidad de columnas categóricas (top 20):")
display(df[cat_cols].nunique().sort_values(ascending=False).head(20).to_frame('valores_unicos'))

# Muestra 5 valores ejemplo por cada categórica (para revisar formato)
for c in cat_cols:
    ej = df[c].dropna().unique()[:5]
    print(f"→ {c}: {ej}")


=== df.info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10132 entries, 0 to 10131
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Numero Cliente                     10132 non-null  int64  
 1   Estado Abandono                    10132 non-null  object 
 2   Edad Cliente                       7736 non-null   float64
 3   Genero                             10132 non-null  object 
 4   Cant. Personas a Cargo             10132 non-null  int64  
 5   Nivel Educativo                    10132 non-null  object 
 6   Estado Civil                       9116 non-null   object 
 7   Rango Ingresos                     10132 non-null  object 
 8   Categoria Tarjeta                  10124 non-null  object 
 9   Antiguedad Cuenta                  10132 non-null  int64  
 10  Total Productos Bancarios          10132 non-null  int64  
 11  Meses Inactivos Ultimo Año         101

,valores_unicos
Nivel Educativo,7
Rango Ingresos,6
Estado Civil,4
Categoria Tarjeta,4
Genero,2
Estado Abandono,2


→ Estado Abandono: ['Cliente Existente' 'Cliente Perdido']
→ Genero: ['Masculino' 'Femenino']
→ Nivel Educativo: ['Secundaria' 'Graduado' 'Sin Educación' 'Desconocido' 'Universidad']
→ Estado Civil: ['Casado' 'Soltero' 'Desconocido' 'Divorciado']
→ Rango Ingresos: ['$60K - $80K' 'Menos de $40K' '$80K - $120K' '$40K - $60K' 'Mas de $120K']
→ Categoria Tarjeta: ['Azul' 'Oro' 'Plata' 'Platino']


<div style="background:#f7f9ff; border:1px solid #e5ecff; border-left:8px solid #4f46e5; border-radius:12px; padding:18px 22px; font-family:'Inter','Segoe UI','Roboto','Helvetica Neue',Arial,sans-serif; line-height:1.55; font-size:15px; color:#0f172a;">

<h2 style="margin:0 0 8px 0;">EDA — Hallazgos iniciales (Tipos &amp; Formatos)</h2>

<p style="margin:0 0 12px 0;">
<strong>Dataset:</strong> 10,132 filas × 21 columnas &nbsp;|&nbsp; 
<strong>Memoria reportada:</strong> ~4.6&nbsp;MB <em>(pequeño; no es cuello de botella. Igual optimizaremos con <code>category</code> y <code>downcast</code>).</em>
</p>

<h3 style="margin:12px 0 6px 0;">1) Identificador</h3>
<ul style="margin:0 0 10px 18px;">
  <li><strong>Numero Cliente</strong>: entero con pinta de <em>ID</em>. No se usará como feature; verificaremos unicidad y lo pondremos como índice.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">2) Variable objetivo</h3>
<ul style="margin:0 0 10px 18px;">
  <li><strong>Estado Abandono</strong> (object): dos clases → <code>Cliente Existente</code> / <code>Cliente Perdido</code>. Mantendremos esta columna legible y crearemos <strong><code>churn</code></strong> binaria: 1 = Perdido, 0 = Existente.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">3) Categóricas (baja cardinalidad)</h3>
<ul style="margin:0 0 10px 18px;">
  <li><code>Genero (2)</code>, <code>Estado Civil (4)</code>, <code>Categoria Tarjeta (4)</code>, <code>Rango Ingresos (6)</code>, <code>Nivel Educativo (7)</code>.</li>
  <li>Convertiremos a <code>category</code> (ahorro de memoria y semántica explícita).</li>
  <li><strong>Ordinales (definir orden de negocio):</strong>
    <ul>
      <li><strong>Categoria Tarjeta</strong>: Azul &lt; Plata &lt; Oro &lt; Platino.</li>
      <li><strong>Rango Ingresos</strong>: Menos de $40K &lt; $40K–$60K &lt; $60K–$80K &lt; $80K–$120K &lt; Más de $120K.</li>
    </ul>
  </li>
  <li><strong>Limpieza:</strong> normalizar mayúsculas/espacios/acentos y unificar <em>Desconocido</em>.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">4) Numéricas</h3>
<ul style="margin:0 0 10px 18px;">
  <li><strong>Edad Cliente</strong>: está en <code>float</code> por nulos; <em>no</em> convertir a <code>int</code> hasta imputar.</li>
  <li><strong>Enteras correctas</strong>: Personas a Cargo, Antigüedad, Total Productos, Meses Inactivos, Frecuencia Contacto, Número Total Transacciones.</li>
  <li><strong>Flotantes correctas</strong>: Límite Crédito, Promedio Crédito Disponible, % Uso Crédito, Cambios Q4/Q1.</li>
  <li><strong>Saldo Pendiente Tarjeta</strong>: hoy entero; si aparecen decimales en futuras fuentes, pasaremos a <code>float</code>.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">5) Columnas “Cambio … Q4 Q1”</h3>
<ul style="margin:0 0 10px 18px;">
  <li>Interpretación esperada: <em>ratio</em>/variación entre Q4 y Q1 ( &gt;1 crece, &lt;1 cae ). Mantener como <code>float</code>. Confirmar definición con negocio.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">6) Nulos relevantes</h3>
<ul style="margin:0 0 10px 18px;">
  <li><strong>Edad Cliente</strong> (~2.4k nulos), <strong>Estado Civil</strong> (~1k nulos), <strong>Categoria Tarjeta</strong> (8 nulos).</li>
  <li>Estrategia baseline: imputación simple (mediana/modo) + categoría “Desconocido”; luego probaremos imputación segmentada/model-based.</li>
</ul>

<h3 style="margin:12px 0 6px 0;">7) Riesgo de fuga temporal (leakage)</h3>
<ul style="margin:0 0 10px 18px;">
  <li>Validar que todas las features correspondan a la ventana de observación y que el label <code>Estado Abandono</code> esté definido en un horizonte posterior.</li>
</ul>

<hr style="border:none; border-top:1px solid #e5ecff; margin:14px 0;">

<h3 style="margin:0 0 6px 0;">Decisiones v1</h3>
<ol style="margin:0 0 10px 18px;">
  <li>Convertir <code>object</code> → <code>category</code> y crear <code>churn</code> binaria (1 = Perdido).</li>
  <li>Definir orden en <strong>Categoria Tarjeta</strong> y <strong>Rango Ingresos</strong>.</li>
  <li>Optimizar memoria con <code>category</code> + <code>downcast</code> en enteros/flotantes; reportar ahorro.</li>
  <li><strong>No</strong> mapear manualmente a 0/1/2… en esta etapa (evitamos orden falso). La codificación será parte del <em>pipeline</em> (One-Hot/Target Encoding).</li>
  <li>Imputación baseline y registro de supuestos.</li>
</ol>

<h3 style="margin:0 0 6px 0;">Próximos pasos inmediatos</h3>
<ul style="margin:0 0 0 18px;">
  <li>Verificar unicidad de <strong>Numero Cliente</strong> y usarlo como índice.</li>
  <li>Limpieza de textos (trim/case/acentos) y unificación de <em>Desconocido</em>.</li>
  <li><em>Casting</em> a <code>category</code>/<code>CategoricalDtype</code> (con orden donde aplique).</li>
  <li>Crear <code>churn</code> binaria y mantener columna original.</li>
  <li>Reporte de <strong>memoria antes/después</strong>.</li>
</ul>

</div>
